In [1]:
import numpy as np
import pandas as pd
import re
import time
from glob import glob
from string import punctuation
from collections import Counter, OrderedDict

import requests
from requests_html import HTMLSession

# from string_grouper import match_strings, match_most_similar, group_similar_strings, StringGrouper
# from sklearn.feature_extraction.text import TfidfVectorizer
# from scipy.sparse import csr_matrix
# import sparse_dot_topn.sparse_dot_topn as sdt

In [2]:
file_list = glob('../data/*.csv')
file_list = sorted(file_list)

responses = pd.read_csv(file_list[-1])
responses.head()

,Timestamp,What's your skin type?,Do you have any known allergies? Or any ingredients you avoid for any reason?,What are your top skin concerns?,What is your current day routine? (Please list your products in order from first to last),What is your current night routine (Please list your products in order from first to last),What type of summer weather will you have?,Any known skin sensitivities?,"Any experience with retinoids? (If so, please tell us your use history)","Any experience with acids? (If so, please tell us your use history, what type, and any known concentrations)","Are you prone to breakouts? If so, how severe are the breakouts and what is your acne treatment history?",Anything else you want the chemists to know about your skin and your skincare routine?,"Can we share your routine learnings (anonymously) on our social media (post, podcast, or insta live shoutout)?"
0,4/30/2020 12:43:32,Combination,None,"Acne, Pigmentation, Fine lines/wrinkles","trader joe's all in one cleanser, bioderma seb...","trader joe's all in one cleanser, Bioderma Seb...","Temperate, a lovely 70F (lucky!)",I broke out from Cosrx snail serum and from Ma...,Differing 0.1% for four years,I prefer toners with acids as masks with AHA s...,I have mild to moderate acne that I have been ...,"I have a super oily forehead, oily nose and c...",Yes
1,4/30/2020 12:45:25,Combination,No allergies,"Acne, Fine lines/wrinkles, Skin Texture (rough...","Biore Charcoal Acne Clearing Cleanser, Kiel's ...",NaN,"Tropical, humid",No.,No.,No.,Yes. They are usually cystic and around my cyc...,I need help.,Yes
2,4/30/2020 12:50:26,Oily,No.,"Acne, Pigmentation","Drunk Elephant Beste No. 9 Jelly Cleanser, pau...","Dermalogica Precleanse, CeraVe Foaming Facial ...",Desert dry heat,No.,No.,I've used 2% BHA on and off; and daily I use a...,"Yes, occasional small whiteheads and blackhead...",I'm Black so hyperpigmentation and skin discol...,Yes
3,4/30/2020 13:02:43,Normal,No,"Fine lines/wrinkles, Skin Texture (roughness, ...","Paula's Choice enriched calming toner, Timeles...","Palmer's facial cleansing oil, Cetaphil gentle...","Tropical, humid",No,Almost through the purging/peeling phase with ...,I've tried (and still own) The Ordinary red pe...,"Not really - a very rare pimple, I tend more o...",I turn 43 in a month and am really trying to g...,Yes
4,4/30/2020 13:08:20,Combination,"Ethylhexyl palmitate, strong sulfates, chemica...","Acne, Skin Texture (roughness, dullness), Other","Osea Ocean Cleanser, Renee Rouleau Elderberry ...","Renee Rouleau Luxe Mint Cleanser, Renee Roulea...","Temperate, a lovely 70F (lucky!)",Overall I have fairly sensitive skin with horm...,"No, but I would like to start to address early...",Azelaic acid has been a good acid for me (not ...,"My acne is adult onset, primarily hormonal, an...","I just moved to Chicago, so lots of temperatur...",Yes


In [3]:
new_columns = [
    'timestamp',
    'skin_type',
    'allergies',
    'top_skin_concern',
    'am_routine',
    'pm_routine',
    'summer_climate',
    'skin_sensitivities',
    'used_retinoids',
    'used_acids',
    'prone_to_breakouts',
    'miscellaneous',
    'permission'
]

responses.columns = new_columns
# responses.drop(columns=['email_address','name'], inplace=True)
responses.head()

,timestamp,skin_type,allergies,top_skin_concern,am_routine,pm_routine,summer_climate,skin_sensitivities,used_retinoids,used_acids,prone_to_breakouts,miscellaneous,permission
0,4/30/2020 12:43:32,Combination,None,"Acne, Pigmentation, Fine lines/wrinkles","trader joe's all in one cleanser, bioderma seb...","trader joe's all in one cleanser, Bioderma Seb...","Temperate, a lovely 70F (lucky!)",I broke out from Cosrx snail serum and from Ma...,Differing 0.1% for four years,I prefer toners with acids as masks with AHA s...,I have mild to moderate acne that I have been ...,"I have a super oily forehead, oily nose and c...",Yes
1,4/30/2020 12:45:25,Combination,No allergies,"Acne, Fine lines/wrinkles, Skin Texture (rough...","Biore Charcoal Acne Clearing Cleanser, Kiel's ...",NaN,"Tropical, humid",No.,No.,No.,Yes. They are usually cystic and around my cyc...,I need help.,Yes
2,4/30/2020 12:50:26,Oily,No.,"Acne, Pigmentation","Drunk Elephant Beste No. 9 Jelly Cleanser, pau...","Dermalogica Precleanse, CeraVe Foaming Facial ...",Desert dry heat,No.,No.,I've used 2% BHA on and off; and daily I use a...,"Yes, occasional small whiteheads and blackhead...",I'm Black so hyperpigmentation and skin discol...,Yes
3,4/30/2020 13:02:43,Normal,No,"Fine lines/wrinkles, Skin Texture (roughness, ...","Paula's Choice enriched calming toner, Timeles...","Palmer's facial cleansing oil, Cetaphil gentle...","Tropical, humid",No,Almost through the purging/peeling phase with ...,I've tried (and still own) The Ordinary red pe...,"Not really - a very rare pimple, I tend more o...",I turn 43 in a month and am really trying to g...,Yes
4,4/30/2020 13:08:20,Combination,"Ethylhexyl palmitate, strong sulfates, chemica...","Acne, Skin Texture (roughness, dullness), Other","Osea Ocean Cleanser, Renee Rouleau Elderberry ...","Renee Rouleau Luxe Mint Cleanser, Renee Roulea...","Temperate, a lovely 70F (lucky!)",Overall I have fairly sensitive skin with horm...,"No, but I would like to start to address early...",Azelaic acid has been a good acid for me (not ...,"My acne is adult onset, primarily hormonal, an...","I just moved to Chicago, so lots of temperatur...",Yes


In [4]:
responses['top_skin_concern'] = responses['top_skin_concern'].str.lower().str.split(', ')
responses['top_skin_concern'][:5]

0            [acne, pigmentation, fine lines/wrinkles]
1    [acne, fine lines/wrinkles, skin texture (roug...
2                                 [acne, pigmentation]
3    [fine lines/wrinkles, skin texture (roughness,...
4    [acne, skin texture (roughness, dullness), other]
Name: top_skin_concern, dtype: object

## using requests-html

In [5]:
class OrderedCounter(Counter, OrderedDict):
    pass

In [6]:
class Cosmetic(HTMLSession):
    '''
    Parent class for connecting to CosDNA.com database.
    Not intended to be used on its own.
    '''
    
    base_url = 'https://incidecoder.com'
    
    # sort_dict = {
    #     'latest': '&sort=date',
    #     'featured': '&sort=featured',
    #     'clicks': '&sort=click',
    #     'reviews': '&sort=review'
    # }
    
    def __init__(self):
        super().__init__()

    def link(self, name=None, url=None):
        self.url = self._check_url(url)

    @property
    def name(self):
        return self.name.replace(' ', '-')

    @staticmethod
    def _check_url(url):
        if 'incidecoder.com' in url:
            return url.lower()
        else:
            return None

In [39]:
class Ingredient(HTMLSession):

    base_url = 'https://incidecoder.com/ingredients/'

    def __init__(self, name=None, url=None):
        self.name = name
        self._name = re.sub(r'\s+', '-', self.name)
        super().__init__()

    def search(self, direct=True):
        if direct:
            try:
                self._r = self.get(self.base_url + self._name)
            except:
                pass
        else:
            self._r = self.get('https://incidecoder.com/search?query=' + self._search_name)

    @property
    def _search_name(self):
        return re.sub(r'\s+', '\+', self.name)

In [40]:
sa = Ingredient('salicylic acid')
sa.search()

In [41]:
sa._r.url

'https://incidecoder.com/ingredients/salicylic-acid'

In [35]:
sa._r.html.find('div#ingredients')[0].links

{'/ingredients/capryloyl-salicylic-acid', '/ingredients/salicylic-acid'}

In [16]:
s = HTMLSession()

In [17]:
r = s.get('https://incidecoder.com/search?query=salicylic+acid')

In [18]:
r.url

'https://incidecoder.com/search?query=salicylic+acid'

In [23]:
r.html.find('div#ingredients')[0].links

{'/ingredients/capryloyl-salicylic-acid', '/ingredients/salicylic-acid'}